# **AI Powered HR Assistant**

By: Abby Krug

11/10/2025

Import/Install Necessary Modules

In [22]:
# install the necessary modules
# %pip install --upgrade huggingface_hub sentence-transformers bitsandbytes
%pip install -qU pypdf
%pip install -qU langchain langchain-community langchain-google-genai langchain-text-splitters langchain-openai
%pip install -qU faiss-cpu
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
# import the necessary modules
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [24]:
# This will load openai and load the API Key from this lab allowing me to use the embeddings and llm

import openai
import os

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [25]:
# I used this when I was trying to use Gemini but I ran into overused API Key calls issue...

#from google import genai
#from getpass import getpass
#api_key = getpass()
#client = genai.Client(api_key=api_key)

**LOAD Documents (PDFs)**

Need to load the information for the nestle HR policy chatbot to enable access from RAG system

In [26]:
# This step will load the pdf for the hr policy that will be loaded into the vector store

loader = PyPDFLoader("nestle_hr_policy_pdf_2012.pdf")
docs = loader.load()

**SPLIT Text into Smaller Chunks**

In [27]:
# This step splits the pdf into smaller text chunks

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
all_txt = doc_splitter.split_documents(docs)

**EMBED all of the chunks**

In [28]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [29]:
# This was when I was using Gemini Embeddings but I was running into issues with my API key...
# Initialize the GoogleGenerativeAIEmbeddings model with the API key
#embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", google_api_key=api_key)

embeddings = OpenAIEmbeddings()

**BUILD Vector Store and Cast it as a Retriever**

In [30]:
# need to import the vector stores before building through langchain
from langchain_community.vectorstores import FAISS

In [31]:
# This step uses FAISS to load the vectorstore and cast it as a retriever

vs = FAISS.from_documents(all_txt, embeddings)
lab=vs.as_retriever()

**DESIGN Prompt Template for LLM**

For this template I included that the LLM should act as a conversational HR Policy Chatbot. The placeholders are context and question. Context will be filled by my RAG function and Question will be filled by the Gradio UI that will ask the user for a question for the chatbot.

In [32]:
# import langchain prompts for templates
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [33]:
# This step will create a template prompt which will be sent to the LLM

temp="""You are a conversational chatbot for question-answering tasks.
You work for Nestle Human Resources.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use one sentence and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt=ChatPromptTemplate.from_template(temp)

output_parser = StrOutputParser()

**LOAD and CONFIGURE LLM Model**

In [16]:
# this cell was for when I tried to use Gemini but was running into API Key issues...

#from langchain_google_genai import ChatGoogleGenerativeAI
#llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

In [34]:
# This step loads the OpenAI llm to use for the chain

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

**SET up pipeline and chain components**

In [35]:
# need to connect the chain together
rag_chain = (
    {"context": lab, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

# this function will execute the chain and return the chatbot response
def rag_chain_fn(message):
    response = rag_chain.invoke(message)
    return response

**Create Gradio UI and ask user for query**

This Gradio UI will have chatbot. The query listed in the textbox will spark the chain to run and then return the chatbot answer to the screen

In [32]:
# need to install gradio
%pip install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Using cached websockets-15.0.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 32.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 5.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 2.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 7.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 6.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.6 MB/s eta 0:00:00:

In [36]:
import gradio as gr

# this is required for the syntax of ChatInterface
def chat_bot(message, history):
    response = rag_chain_fn(message)
    return response

# trigger function that invokes rag_chain
g = gr.ChatInterface(
   fn = chat_bot,
   title = "NESTLE HR Chatbot",
   description = "Ask questions about the Nestle HR Policy",
   theme = "soft",
)

if __name__ == "__main__":
  g.launch(share=True)

/voc/work/.local/lib/python3.10/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://1524eab57ee9bbea9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
